<a href="https://colab.research.google.com/github/Yying-Gis/York_Case_Study/blob/main/CASE_STUDY_Simple_Identification_of_Areas_with_Highest_Community_Resource_Density.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CASE STUDY: Simple Identification of Areas with Highest Community Resource Density**
## **Aim**
*  The aim of this case study is to identify areas within the city of York that have the highest density of community resources, including educational institutions, medical facilities, public infrastructure, social services, and cultural amenities.

*  By visualizing and quantifying the distribution of these resources using spatial analysis and heat maps, we aim to highlight disparities or imbalances in community resource allocation and provide recommendations for the best living locations in York.

## **Location: York, UK**

## **Source:**
###  **1. Educational resources**

        *   Libraries: https://www.data.gov.uk/dataset/20800028-b103-4693-aab7-265665e7226f/libraries5

              *  Published by: City of York Council  
              *  Last updated: 13 February 2020

        *   Primary Schools' Location: http://data.cyc.opendata.arcgis.com/datasets/7dc1e92e14ed43148a1ff542b8820019_0.geojson
        
              *  Published by: City of York Council       
              *  Last updated: September 15, 2015

        *   Secondary Schools' Location: http://data.cyc.opendata.arcgis.com/datasets/c431b649ce424799af7316eb5db4394b_1.geojson
        
              *  Published by: City of York Council       
              *  Last updated: September 15, 2015


###  **2. Public infrastructure**

        *   Street Lighting: https://www.data.gov.uk/dataset/baf1c264-7160-444b-a4e3-0fe25c5b1e11/street-lighting
          *  Published by: City of York Counci      
          *  Last updated: 13 July 2024

        *   Concentration of transportation/stop: https://www.data.gov.uk/dataset/ff93ffc1-6656-47d8-9155-85ea0b8f2251/naptan/stop.csv
        
              *  Published by: Department for Transport    
              *  Last updated: 23 August 2022*
###  **3. Medical facilities**

        *   Pharmacies: https://www.data.gov.uk/dataset/f16ee625-a2ec-402b-bfb8-216a421e2546/pharmacies2     
              *  Published by: City of York Council
              *  Last updated: 02 February 2024

        *   GP Surgeries: https://www.data.gov.uk/dataset/5490d87f-aacf-4f4e-9607-06a33a09b78b/gp-surgeries    
              *  Published by: City of York Council
              *  Last updated: 02 February 2024

        *   Hospital: osmnx
              *  source from: osmnx- python package from OpenStreetMap
              *  Last updated: June 2025

###  **4. ART & cultural facilities**

        *   Conservation Areas: https://opendata.arcgis.com/datasets/3282e030bf2b4768a05984de8d3a9204_29.geojson     
              *  Published by: City of York Council
              *  Last updated: September 14, 2015

###  **5. Social services**

        *   Community-venuesosmnx
              *  source from: osmnx- python package from OpenStreetMap
              *  Last updated: June 2025


  ***York Boundaries: https://data.yorkopendata.org/dataset/yorks-boundaries/resource/cd3d05fd-3464-4599-bf92-94284c1b2497?view_id=a0588a91-de41-4ab1-8f9c-856f11994b69***

              *  Published by: City of York Council
              *  Last updated: July 29, 2015



# **1. Heat Map of Community Resource Density in York**

**# Install required libraries**

In [13]:
!pip install geopandas
import geopandas as gpd
!pip install folium
from folium.plugins import HeatMap
import folium
!pip install osmnx
import osmnx as ox
!pip install pandas
import pandas as pd
from shapely.geometry import Point
from branca.element import Figure

**Load York’s administrative boundary and data**

In [14]:
# Load York’s administrative boundary
york_Ward_Boundaries_path = 'http://data.cyc.opendata.arcgis.com/datasets/8c16fc70e4e0452ea750361ea0e19d4d_8.geojson'
df_york_Ward_Boundaries = gpd.read_file(york_Ward_Boundaries_path)


# Load educational facilities data
df_libraries = gpd.read_file('https://raw.githubusercontent.com/Yying-Gis/York_Case_Study/refs/heads/main/Data/Libraries.geojson')
df_Primary_Schools = gpd.read_file('http://data.cyc.opendata.arcgis.com/datasets/7dc1e92e14ed43148a1ff542b8820019_0.geojson')
df_Secondary_Schools = gpd.read_file('http://data.cyc.opendata.arcgis.com/datasets/c431b649ce424799af7316eb5db4394b_1.geojson')

# Load public infrastructure data
df_Street_Lighting = gpd.read_file('https://opendata.arcgis.com/datasets/2cbf2c90548b4d6aae92c42917531d96_3.geojson')
df_stop = pd.read_csv("https://media.githubusercontent.com/media/Yying-Gis/York_Case_Study/dd4c5136344ff96e489a014038e229342e21faff/Stops.csv", on_bad_lines='skip', sep=",",low_memory=False)

# Load medical facilities using OSM and GeoJSON
df_pharmacies = gpd.read_file('https://raw.githubusercontent.com/Yying-Gis/York_Case_Study/refs/heads/main/Data/Pharmacies.geojson')
df_GP_Surgeries = gpd.read_file('https://raw.githubusercontent.com/Yying-Gis/York_Case_Study/c271e7a314d654fbf4c3a4bb51740134954a6574/Data/GP_Surgeries.geojson')
df_hospitals = ox.features_from_place("York, UK", tags={'amenity':"hospital"})

# Load art & cultural facility data
df_conservation_areas = gpd.read_file('https://opendata.arcgis.com/datasets/3282e030bf2b4768a05984de8d3a9204_29.geojson')

# Load social service data
df_community_facilities = ox.features_from_place("York, UK", tags={"amenity": "community_centre"})
df_community_facilities.head()
print(df_GP_Surgeries)


    OBJECTID                       Group_                          Address_1  \
0          1  Old School Medical Practice        Old School Medical Practice   
1          2                                       Unity Health, York Campus   
2          3                                                    Unity Health   
3          4         Independant surgerys  Jorvik Gillygate Medical Practice   
4          5                                                    Unity Health   
5          6         Priory Medical Group              Priory Medical Centre   
6          7         Priory Medical Group                  Rawcliffe Surgery   
7          8         Priory Medical Group         Clementhorpe Health Centre   
8          9         Priory Medical Group             Lavender Grove Surgery   
9         10         Priory Medical Group              Heworth Green Surgery   
10        11         Priory Medical Group                    Fulford Surgery   
11        12         Priory Medical Grou

**Setup Methods**


1.  Postcode to coordinate
2.  Address to coordinate
3.  Setup s method-# Add latitude/longitude columns for each point feature in educational datasets



In [15]:
 #unctions included to support coordinate lookup from address or postcode
def postcode_to_coords(postcode):
    try:
        location = geocode(f"{postcode}, UK", exactly_one=True)
        if location:
            return (location.longitude, location.latitude)
        return (None, None)
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Postcode {postcode} Conversion failed: {str(e)}")
        return (None, None)
    except Exception as e:
        print(f"Postcode {postcode} occurs error: {str(e)}")
        return (None, None)

#address to coordinate
def address_to_coords(address):
    try:
        location = geocode(f"{address}, York, UK", exactly_one=True)
        if location:
            return (location.longitude, location.latitude)
        return (None, None)
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Address {address} Conversion failed: {str(e)}")
        return (None, None)
    except Exception as e:
        print(f"Address {address} occurs error: {str(e)}")
        return (None, None)

# Coordinate extraction helper function
def extract_lon_lat(df):
    def get_coords(geom):
        if geom.geom_type == 'Point':
          return geom.x, geom.y
        ## Convert Polygon to single Points
        elif geom.geom_type in ('Polygon', 'MultiPolygon'):
          return geom.centroid.x, geom.centroid.y
        ## Convert MultiPoints to single Points
        elif geom.geom_type == 'MultiPoint':
          return geom.geoms[0].x, geom.geoms[0].y  #get the first point
        else:
          return None, None

    df['Longitude'], df['Latitude'] = zip(*df['geometry'].map(get_coords))
    #print(df['Longitude'])
    return df.dropna(subset=['Longitude', 'Latitude'])

**Add latitude/longitude columns for each point feature in educational datasets**

In [16]:
extract_lon_lat(df_libraries)
df_libraries.head() # geometry | MULTIPOINT ((-1.11515 53.95331))
extract_lon_lat(df_Primary_Schools)
df_Primary_Schools.head()
extract_lon_lat(df_Secondary_Schools)
#df_Secondary_Schools.head()

# Merge libraries, primary, and secondary schools into one DataFrame
combined_Education_Resources_df = pd.concat([df_libraries,df_Primary_Schools,df_Secondary_Schools], ignore_index=True)
combined_Education_Resources_df.head()

,OBJECTID,NAME,ADDRESS,TYPE,LV_LABEL,PHONE,EMAIL,EXPLORE_YORK_URL,geometry,Longitude,Latitude,OBJECTID_1,SCHNAME,DFESNO,WARD,LV_DETAILS,WEBSITE,Website4,LABEL
0,791.0,Rowntree Park Reading Cafe,"Rowntree Park Lodge, Richardson Street, YO23 1JU",Reading Cafe,Rowntree Park Reading Cafe,(01904) 551489,rowntreepark@exploreyork.org.uk,https://www.exploreyork.org.uk/cafes/rowntree-...,POINT (-1.08274 53.94893),-1.082736,53.948933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,792.0,Huntington Library,"Garth Road, Huntington, York, YO32 9QJ",Community Library,Huntington Library,(01904) 552 669,huntington@exploreyork.org.uk,https://www.exploreyork.org.uk/huntington-libr...,POINT (-1.05593 53.99553),-1.055932,53.995527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,793.0,York Explore Library,"Museum Street, York, YO17DS",Explore Centre,York Explore Library,(01904) 552 828,york@exploreyork.org.uk,https://www.exploreyork.org.uk/york-explore/,POINT (-1.08621 53.96144),-1.086207,53.961439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,794.0,Tang Hall Library,"The Centre @ Burnholme, Mossdale Avenue, York,...",Explore Centre,Tang Hall Explore Library,(01904) 552 655\r\n,tanghall@exploreyork.org.uk,https://www.exploreyork.org.uk/tang-hall-explore/,POINT (-1.04795 53.96458),-1.047945,53.964583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,795.0,Strensall Library,"19 The Village, Strensall, York, YO32 5XS",Community Library,Strensall Library,(01904) 552 677,strensall@exploreyork.org.uk,https://www.exploreyork.org.uk/strensall-library/,POINT (-1.03579 54.03995),-1.035786,54.039952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Same logic applied to medical, infrastructure, and social data below**

In [17]:
#Medical
extract_lon_lat(df_pharmacies)
extract_lon_lat(df_GP_Surgeries)
extract_lon_lat(df_hospitals)
#combine df
combined_Medical_Resources_df = pd.concat([df_pharmacies,df_GP_Surgeries,df_hospitals], ignore_index=True)
#print(combined_Medical_Resources_df['Latitude'], combined_Medical_Resources_df['Longitude'])


In [18]:
# infrastructure
extract_lon_lat(df_Street_Lighting)
#df_Street_Lighting.head()
#print(len(df_stop))

#convert from csv to gdf
geometry = [Point(xy) for xy in zip(df_stop['Longitude'], df_stop['Latitude'])]
gdf_stop = gpd.GeoDataFrame(df_stop, geometry=geometry, crs="EPSG:4326")

# Filter transportation stops to only those inside York's boundary
df_stop_within_Boundaries = gpd.sjoin(gdf_stop, df_york_Ward_Boundaries, how="inner",predicate="intersects")
#keep column as csv
columns_to_keep = list(gdf_stop.columns)
#Check the number of transportation stops in York
print(len(df_stop_within_Boundaries))

# Merged Street Lighting and transportation stops into one DataFrame
combined_Public_infrastructure_df = pd.concat([df_Street_Lighting,df_stop_within_Boundaries], ignore_index=True)
#print(df_stop_within_Boundaries['Latitude'], df_stop_within_Boundaries['Longitude'])

1272


In [19]:
#ART & cultural facilities
extract_lon_lat(df_conservation_areas)
#Social services
extract_lon_lat(df_community_facilities)
# Combine all resource types into thematic categories
combined_dataframes = [combined_Education_Resources_df,combined_Medical_Resources_df,df_conservation_areas,df_community_facilities, combined_Public_infrastructure_df]

Reproject spatial data to a common CRS for accurate spatial analysis (EPSG:4326 for web maps)

In [20]:
World_Grid = "EPSG:4326"
# For loop - convert CRS to EPSG:4326 for map creation
def convert_to_bng_4326(df, target_crs="EPSG:4326"):
    if df.crs != target_crs:
        df = df.to_crs(target_crs)
    return df
world_data_frames = [convert_to_bng_4326(df) for df in combined_dataframes]

#list all dataframe's CRS
for i, df in enumerate(world_data_frames, 1):
    df_type= type(df)
    crs = df.crs if hasattr(df, 'crs') else "Not a GeoDataFrame"
    print(f"df{i} CRS:", crs," type:",df_type )

df1 CRS: EPSG:4326  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df2 CRS: EPSG:4326  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df3 CRS: EPSG:4326  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df4 CRS: epsg:4326  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df5 CRS: EPSG:4326  type: <class 'geopandas.geodataframe.GeoDataFrame'>


**Generate interactive heatmaps for each category and an overall map**

In [21]:
combined_facility_data = pd.concat(world_data_frames, ignore_index=True)
map_center_latitude = combined_facility_data['Latitude'].mean()
map_center_longitude = combined_facility_data['Longitude'].mean()

gradient = {0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}

basemap_facility = folium.Map(location=[(map_center_latitude),(map_center_longitude)],zoom_start=12, tiles='OpenStreetMap')

fg_edu = folium.FeatureGroup(name='Education').add_to(basemap_facility)
HeatMap(combined_Education_Resources_df[['Latitude', 'Longitude']], radius=10, gradient=gradient).add_to(fg_edu)

fg_P_inf = folium.FeatureGroup(name='Public infrastructure').add_to(basemap_facility)
HeatMap(combined_Public_infrastructure_df[['Latitude', 'Longitude']], radius=10, gradient=gradient).add_to(fg_P_inf)

fg_Med = folium.FeatureGroup(name='Medical facilities').add_to(basemap_facility)
HeatMap(combined_Medical_Resources_df[['Latitude', 'Longitude']], radius=10, gradient=gradient).add_to(fg_Med)

fg_SS= folium.FeatureGroup(name='Social services').add_to(basemap_facility)
HeatMap(df_community_facilities[['Latitude', 'Longitude']], radius=10, gradient=gradient).add_to(fg_SS)

fg_cultural = folium.FeatureGroup(name='ART & cultural facilities').add_to(basemap_facility)
HeatMap(df_conservation_areas[['Latitude', 'Longitude']], radius=10, gradient=gradient).add_to(fg_cultural)

fg_all = folium.FeatureGroup(name='All Facilities').add_to(basemap_facility)
HeatMap(combined_facility_data[['Latitude', 'Longitude']], radius=10, gradient=gradient).add_to(fg_all)

## Add administrative boundaries for reference
fg_ward_boundary = folium.FeatureGroup(name='ward boundary').add_to(basemap_facility)
folium.GeoJson(df_york_Ward_Boundaries,style_function=lambda x: {'color': 'black', 'weight': 2, 'fillOpacity': 0}, tooltip=folium.GeoJsonTooltip(fields=['NAME', 'CODE'], style=("font-size: 20px;")) ).add_to(fg_ward_boundary)

folium.LayerControl().add_to(basemap_facility)
basemap_facility.save('/content/drive/MyDrive/UK_DATA/York/York-Highest Community Resource Density.html')
basemap_facility

In [22]:
print(df_york_Ward_Boundaries)

    OBJECTID                              NAME AREA_CODE  \
0         22                    Strensall Ward       UTW   
1         23                    Wheldrake Ward       UTW   
2         24         Fulford & Heslington Ward       UTW   
3         25              Rural West York Ward       UTW   
4         26                 Copmanthorpe Ward       UTW   
5         27                 Bishopthorpe Ward       UTW   
6         28  Rawcliffe & Clifton Without Ward       UTW   
7         29         Osbaldwick & Derwent Ward       UTW   
8         30              Heworth Without Ward       UTW   
9         31    Huntington & New Earswick Ward       UTW   
10        32            Haxby & Wigginton Ward       UTW   
11        33     Dringhouses & Woodthorpe Ward       UTW   
12        34                    Westfield Ward       UTW   
13        35                   Micklegate Ward       UTW   
14        36                   Fishergate Ward       UTW   
15        37                    Hull Roa

# **2. York- Highest Community Resource Density**
1. Calculate the total area of the York boundary.
2. Check the density of each resource point.

In [23]:
# Convert CRS to British National Grid for area and density calculation (EPSG:27700)
British_National_Grid = "EPSG:27700"
def convert_to_bng_27700(df, target_crs="EPSG:27700"):
    if df.crs != target_crs:
        df = df.to_crs(target_crs)
    return df
#Combine all data frame into one List
dataframes = [df_libraries, df_Primary_Schools, df_Secondary_Schools, df_Street_Lighting, df_stop_within_Boundaries, df_pharmacies, df_GP_Surgeries, df_hospitals, df_conservation_areas, df_community_facilities]
British_National_data_frames = [convert_to_bng_27700(df) for df in dataframes]

#check CRS before calculate density
for i, df in enumerate(British_National_data_frames, 1):
    df_type= type(df)
    crs = df.crs if hasattr(df, 'crs') else "Not a GeoDataFrame"
    print(f"df{i} CRS:", crs," type:",df_type )

df_york_Ward_Boundaries = convert_to_bng_27700(df_york_Ward_Boundaries)

#get the total area of df_York_boundaries
York_total_area = df_york_Ward_Boundaries.geometry.area.sum() / 1_000_000   # Convert to square kilometers
print(f"Total area of York: {York_total_area:.2f} km²") #

def calculate_density(gdf, area):
    if area <= 0:
        return 0
    return len(gdf) / area

#list for calculate and print out
resources = [
    ("libraries", df_libraries),
    ("primary schools", df_Primary_Schools),
    ("secondary schools", df_Secondary_Schools),
    ("Street Lighting",df_Street_Lighting),
    ("public transport stop", df_stop_within_Boundaries),
    ("pharmacies",df_pharmacies),
    ("GP Surgeries", df_GP_Surgeries),
    ("Hospitals", df_hospitals),
    ("conservation_ areas",df_conservation_areas),
    ("community facilities",df_community_facilities)
]

# Compute resource density = total number of resources / total city area
for name, df in resources:
    density = calculate_density(df, York_total_area)
    print(f"density of {name} in York: {density:.2f} Units/km²")





df1 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df2 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df3 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df4 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df5 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df6 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df7 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df8 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df9 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
df10 CRS: EPSG:27700  type: <class 'geopandas.geodataframe.GeoDataFrame'>
Total area of York: 272.03 km²
density of libraries in York: 0.06 Units/km²
density of primary schools in York: 0.19 Units/km²
density of secondary schools in York: 0.04 Units/km²
density of Street Lighting in York: 81.68 Units/km²
density of public transport stop in Y